In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import MobileNetV2
from keras.optimizers import Adam

In [ ]:
import zipfile
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(os.listdir("drive/MyDrive/Plant/"))

zip_file = 'drive/MyDrive/Plant/ds_sep.zip'
# zip_file = 'drive/Plant/dataset.zip'
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

print(os.listdir())

['dataset.zip', 'ds2.zip', 'ds_sep.zip']
['.config', 'drive', 'ds_sep', 'sample_data']


In [ ]:
# Пути к данным
train_dir = 'ds_sep/train'
val_dir = 'ds_sep/val'
test_dir = 'ds_sep/test'

# Настройки
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 64
nb_train_samples = 4570
nb_validation_samples = 1407
nb_test_samples = 1062

In [ ]:
MN2_net = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
MN2_net.trainable = False
MN2_net.summary()

In [ ]:
from keras.layers import GlobalAveragePooling2D 

model = Sequential()
model.add(MN2_net)
model.add(GlobalAveragePooling2D())
model.add(Dense(196, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', # Adam(lr=1e-5),
              metrics=['accuracy', TopKCategoricalAccuracy(k=5)])

In [ ]:
from keras.metrics import TopKCategoricalAccuracy

model.compile(loss='categorical_crossentropy',
              optimizer='adam', # Adam(lr=1e-5), 
              metrics=['accuracy', TopKCategoricalAccuracy(k=5)])


In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=30, zoom_range=0.5,
                                   horizontal_flip=True, vertical_flip=True)

datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    )

Found 4570 images belonging to 196 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1407 images belonging to 196 classes.


In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1062 images belonging to 196 classes.


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=50,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
71/71 [==============================] - 53s 709ms/step - loss: 4.7388 - accuracy: 0.0995 - top_k_categorical_accuracy: 0.2374 - val_loss: 2.5225 - val_accuracy: 0.4546 - val_top_k_categorical_accuracy: 0.7292
Epoch 2/50
71/71 [==============================] - 50s 698ms/step - loss: 2.5400 - accuracy: 0.4279 - top_k_categorical_accuracy: 0.6912 - val_loss: 1.8220 - val_accuracy: 0.5759 - val_top_k_categorical_accuracy: 0.8311
Epoch 3/50
71/71 [==============================] - 50s 700ms/step - loss: 1.9462 - accuracy: 0.5309 - top_k_categorical_accuracy: 0.7993 - val_loss: 1.4419 - val_accuracy: 0.6719 - val_top_k_categorical_accuracy: 0.8891
Epoch 4/50
71/71 [==============================] - 50s 700ms/step - loss: 1.5886 - accuracy: 0.6300 - top_k_categorical_accuracy: 0.8602 - val_loss: 1.2615 - val_accuracy: 0.6949 - val_top_k_categorical_accuracy: 0.9182
Epoch 5/50
71/71 [==============================] - 50s 701ms/step - loss: 1.4195 - accuracy: 0.6564 - top_k_categor

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [ ]:
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))
print(scores)

Точность на тестовых данных: 83.89%
[0.6330968141555786, 0.8388671875, 0.95703125]


In [ ]:
model.save("drive/MyDrive/result/MobileNetV2_model.h5")

model.save_weights('drive/MyDrive/result/MobileNetV2_weights.h5')
